# Estruturas Criptográficas
## Trabalho Prático 3 - Exercício 1
### José de Matos Moreira - PG53963
### Pedro Freitas - PG52700

## Enunciado do problema
No capítulo 5 dos apontamentos, é descrito o chamado **Hidden Number Problem**. No capítulo 8 dos apontamentos, é discutido um artigo de **Nguyen & Shparlinsk**, onde se propõem reduções do **HNP** a problemas difíceis em reticulados. Neste trabalho, pretende-se construir, com a ajuda do **Sagemath**, uma implementação da solução discutida nos apontamentos para resolver o **HNP** com soluções aproximadas dos problemas em reticulados.

## Resolução

Todo o código foi desenvolvido com base na íntegra dos apontamentos referidos no enunciado do problema. Desta forma, passa-se, então, a explicar as diversas funções utilizadas durante a realização do exercício proposto:
* **msb_k**: função que extrai, sob a forma de um inteiro positivo, os k *bits* mais significativos do argumento
* **generate_pairs**: algoritmo que gera os pares que obedecem à regra u​<sub>i</sub> ​= msb​<sub>k</sub>​​(⌊s × x<sub>i</sub>​​⌋​<sub>p</sub>​​) para todo i = 1..n
* **build_lattice**: algoritmo que produz o reticulado, a partir da matriz geradora G​' ∈ Q<sup>​m × m</sup>, com m = n + 2, sendo n a dimensão dos pares gerados
* **reduce_lattice**: função responsável por aplicar a redução ao reticulado
* **find_secret**: algoritmo com a capacidade de recuperar o segredo

In [24]:
def msb_k(y, B):
    return y // B


def generate_pairs(n, p, s, B):
    pairs = []
    for _ in range(n):
        x_i = randint(0, p - 1)
        u_i = msb_k((s * x_i) % p, B)
        pairs.append((x_i, u_i))

    return pairs


def build_lattice(xs, us, A, n, p, lmbda, B):
    basis_vectors = []

    for i in range(n):
        vector = [0] * (n + 2)
        vector[i] = p
        basis_vectors.append(vector)

    basis_vectors.append(xs + [A] + [0])

    M = lmbda * p
    basis_vectors.append([-B * u for u in us] + [0] + [M])

    return Matrix(QQ, basis_vectors)


def reduce_lattice(lattice):
    return lattice.LLL()


def find_secret(reduced_lattice, lmbda, p):
    return (reduced_lattice[-1][-2] * lmbda).ceil() % p

### Testes de aplicação

Para efeitos de teste, desenvolveu-se a função **solve_HNP** que, agregando todas as funções anteriormente descritas, resolve o problema **HNP**. Acrescentam-se, também, as regras às quais a função obedece de forma a que a resolução aconteça da forma esperada:
* **p** é um valor primo
* **k** é menor que log<sub>2</sub>p
* **s** obedece à regra s ≠ 0 ∈ Z<sub>p</sub>
* **lmbda** obedece a λ ≡ 2<sup>k</sup>
* **A** é da forma A ≡ 1 / λ
* **B** obedece a B ≡ p / λ

In [25]:
import math

def solve_HNP(d):
    p = next_prime(2 ** d)
    k = floor(sqrt(d) + log(d, 2))
    lmbda = 2 ** k
    A = 1 / lmbda
    B = p / lmbda
    n = floor(2 * sqrt(d))
    s = randint(1, p - 1)
    print('s:', s)

    pairs = generate_pairs(n, p, s, B)
    lattice = build_lattice([x for x, _ in pairs], [u for _, u in pairs], A, n, p, lmbda, B)
    reduced_lattice = reduce_lattice(lattice)
    recovered_s = find_secret(reduced_lattice, lmbda, p)
    print('recovered s:', recovered_s)

    if (s == recovered_s):
        print('HNP solved!')

    else:
        print("Couldn't solve HNP!")

Apresentam-se, assim, três diferentes testes efetuados com diferentes valores de **d**.

In [26]:
solve_HNP(256)

s: 107747632515893736307003412558112463839274318006488029091570415930549959225371
recovered s: 107747632515893736307003412558112463839274318006488029091570415930549959225371
HNP solved!


In [27]:
solve_HNP(512)

s: 2765633407280853479224076066139432473693032170019158495608191794384287061390369680914963773797926048901299836669700227967644999017162220372923763585975361
recovered s: 2765633407280853479224076066139432473693032170019158495608191794384287061390369680914963773797926048901299836669700227967644999017162220372923763585975361
HNP solved!


In [28]:
solve_HNP(1024)

s: 124091559600847266358506865858783322335608668942855151191339580228966464671480829252682736400346337696169035430626006030595659866613628472370523683619475634117534013100252031467071078881632173197034369084954575867639205396038422790197189835661794530430900377556597012144051669581481862433600582659506243558219
recovered s: 124091559600847266358506865858783322335608668942855151191339580228966464671480829252682736400346337696169035430626006030595659866613628472370523683619475634117534013100252031467071078881632173197034369084954575867639205396038422790197189835661794530430900377556597012144051669581481862433600582659506243558219
HNP solved!
